# <u><p style="text-align: center;">Dataframes</p></u>

### Learning goals  
Students will:  
* Learn about Spark Dataframes

### Background

As we have seen, RDDs are the building blocks of Spark. RDDs have several advantages but in some cases their use can be problematic. Such cases can occur because Spark does not optimize transformations when we perform them directly to RDDs. Another example is that working with RDDs in some programming languages (including Python) can lead to poor performance. Also, transformation chains with RDDs can be difficult to comprehend since they show how the result will be achieved but not what the result will be.

Spark **DataFrames** were conceived to overcome the aforementioned problems. Similar to RDDs, DataFrames are distributed collections of data. The difference is that DataFrames provide a high-level abstraction over RDDs that allows us to use a query language to manipulate data. This abstraction is a logical plan that represents data and a schema. The logical plan is converted to a physical plan for execution. This conversion brings us closer to **what** we want to do rather than **how** we have to do it, because we let Spark figure out the most efficient way to carry out the operations. Dataframes are generally faster than RDDs, and they perform the same no matter what programming language we use with Spark.

### Code examples

Before proceeding to the examples, we are going to initialize Spark:

In [ ]:
from pyspark import SparkContext
import os

#'swan_spark_conf' is a configuration provided by a plugin for Jupyter. We further extend this configuration with proxy settings.
swan_spark_conf = swan_spark_conf.setAll([('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')])

#instantiate a SparkContext object with our configuration
sc = SparkContext.getOrCreate(conf=swan_spark_conf)

and create an SQLContext which will help us create DataFrames:

In [ ]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

#### Example 1:

In our first example we are going to create a DataFrame 'manually' which will contain the data of our cows. The data consist of the name, breed and weight of each cows. From those data we would like to have an overwview of the weight and population of each breed.

So, first we create our DataFrame:

In [ ]:
cowsDF = sqlc.createDataFrame([("Joel", "Angus", 450), 
                               ("Marcia", "Belted Galloway", 320),
                               ("Gregor", "Hereford", 390),
                               ("Anne", "Angus", 400),
                               ("Ravi", "Belted Galloway", 250),
                               ("Marcia", "Belted Galloway", 320)],
                              ("Name", "Breed", "Weight"))

and examine it using the function `show`:

In [ ]:
cowsDF.show()

We notice that 'Marcia' has been entered twice in our records so we have to clean our data before we proceed. We can delete duplicate records with the `dropDuplicates` function:

In [ ]:
cowsDF = cowsDF.dropDuplicates(["Name", "Breed", "Weight"])
cowsDF.show()

Next, we would like to inspect the weight of our cows from lighter to heavier. To do this we order our DataFrame using the `orderBy` function:

In [ ]:
orderedDF = cowsDF.orderBy("Weight")
orderedDF.show()

Now that we have an overview of the weight, we would like to order the weights based on breed. This can be done by combining `groupBy` with `orderBy`:

In [ ]:
groupedDF = cowsDF.orderBy(['Breed','Weight'])
groupedDF.show()

Finally, we would like to count how many cows of each breed we have:

In [ ]:
countDF = cows.groupBy("Breed").count()
countDF.show()

#### Example 2:

DataFrames provide a convenient way to work with tabular data. In this example, we are going to read a file with Spark and convert into a DataFrame. The file contains the minimum and maximum daily temperatures for the years 2010-2015 in De Bilt, Netherlands. 

Then, we are going to find the minimum and maximum temperatures that occured during these years and also count how many days the temperature was below 0 $^\text{o}C$.

So, the first step is to load the data into a DataFrame:

In [ ]:
dataDF = sqlc.read.csv("/home/jovyan/datasets/knmi-debilt.csv", header=True, inferSchema=True)

and then examine how the data look like:

In [ ]:
dataDF.show()

Dates are formatted as YYYYMMDD, temperatures are in Celcius degrees.

Next, to find the minimum and maximum temperatures we are going to use **aggregations** over the DataFrame. We can perform aggregations by using the `agg` function. The parameters of `agg` are expressions that indicate the aggregation that we want to perform. To find the maximum temperature a possible solution is:

In [ ]:
from pyspark.sql import functions as F

result = dataDF.agg(F.max("Tmax")) #notice that Tmax is the name of the column
result.show()

and similarly for the minimum:

In [ ]:
result = dataDF.agg(F.min("Tmin")) #notice that Tmin is the name of the column
result.show()

Now, to find how many days the temperature was below 0 $^\text{o}C$, we are first going to keep only the days with the required temperature by using the `filter` function:

In [ ]:
below_zeroDF = dataDF.filter(F.col("Tmin") < 0)

followed by the `count` function:

In [ ]:
below_zeroDF.count()

#### Example 3:

When working with DataFrames we can also write SQL queries against the DataFrame. Using the previous dataset we are going to preserve only the rows of the year 2012. To do this we are first creating a temporary view of the data:

In [ ]:
dataDF.createOrReplaceTempView("data_view")

and then query it using sql syntax:

In [ ]:
only_2012DF = sqlc.sql("SELECT Date, Tmin, Tmax FROM data_view WHERE SUBSTRING(Date,1,4) == 2012")
only_2012DF.show()

<span style="display:none" id="question1">W3sicXVlc3Rpb24iOiAiU3BhcmsgRGF0YUZyYW1lIG9wZXJhdGlvbnMgYXJlIG9wdGltaXplZCBieSBTcGFyay4iLCAidHlwZSI6ICJtdWx0aXBsZV9jaG9pY2UiLCAiYW5zd2VycyI6IFt7ImNvZGUiOiAiVHJ1ZSIsICJjb3JyZWN0IjogdHJ1ZX0sIHsiY29kZSI6ICJGYWxzZSIsICJjb3JyZWN0IjogZmFsc2V9XX1d</span>

<span style="display:none" id="question2">W3sicXVlc3Rpb24iOiAiU3BhcmsgRGF0YUZyYW1lcyBhcmUgYnVpbHQgb24gdG9wIG9mIFJERHMuIiwgInR5cGUiOiAibXVsdGlwbGVfY2hvaWNlIiwgImFuc3dlcnMiOiBbeyJjb2RlIjogIlRydWUiLCAiY29ycmVjdCI6IHRydWV9LCB7ImNvZGUiOiAiRmFsc2UiLCAiY29ycmVjdCI6IGZhbHNlfV19XQ==</span>

<span style="display:none" id="question3">W3sicXVlc3Rpb24iOiAiQ2hvb3NlIHRoZSBjb3JyZWN0IGFuc3dlcnM6IiwgInR5cGUiOiAibXVsdGlwbGVfY2hvaWNlIiwgImFuc3dlcnMiOiBbeyJjb2RlIjogIlNwYXJrIERhdGFGcmFtZXMgYXJlICAgIG5vbi1kaXN0cmlidXRlZCBjb2xsZWN0aW9ucyBvZiBkYXRhLiIsICJjb3JyZWN0IjogZmFsc2UsICJmZWVkYmFjayI6ICJUaGV5IGFyZSBkaXN0cmlidXRlZC4ifSwgeyJjb2RlIjogIldlIGNhbiB1c2UgU1FMIHF1ZXJpZXMgIGRpcmVjdGx5IHdpdGggRGF0YUZyYW1lcy4iLCAiY29ycmVjdCI6IHRydWV9LCB7ImNvZGUiOiAiVGhlIHBlcmZvcm1hbmNlIHdlIGdldCAgd2hlbiB1c2luZyBEYXRhRnJhbWVzIGlzIHByb2dyYW1taW5nIGxhbmd1YWdlIGRlcGVuZGVudC4iLCAiY29ycmVjdCI6IGZhbHNlLCAiZmVlZGJhY2siOiAiRGF0YUZyYW1lcyBoYXZlIHRoZSBzYW1lIHBlcmZvcm1hbmNlIHJlZ2FyZGxlc3Mgb2YgdGhlIGxhbmd1YWdlIHVzZWQuIn0sIHsiY29kZSI6ICJXaGVuIHdvcmtpbmcgd2l0aCAgICAgICBEYXRhRnJhbWVzIHdlIGhhdmUgdG8gY2FyZWZ1bGx5IHRoaW5rIHRoZSBvcmRlciBvZiB0aGUgb3BlcmF0aW9ucyB0aGF0IHdlIHdhbnQgdG8gYXBwbHkuIiwgImNvcnJlY3QiOiB0cnVlfV19XQ==</span>

### Quiz

#### Q1:

In [ ]:
from jupyterquiz import display_quiz

display_quiz("#question1")

#### Q2:

In [ ]:
display_quiz("#question2")

#### Q3:

In [ ]:
display_quiz("#question3")

### More advanced examples:

#### Example A1:

Similarly to RDDs, DataFrame operations are divided into transformations and actions. Results will not be computed until we call an action. When we call an action all the previous transformations up to this action are performed and we get a result. However, during this procedure no intermmediate state of the DataFrame is stored. So, if we need one of these states, Spark has to start the computations from the beggining.

Such an example is depicted below. Each circle represents a transformation that a DataFrame has to go through:

<img src="transformation_chain.png" alt="drawing" width="170"/>

After transformation B, the transformation chain splits into two branches. We can calculate the chain A->B->C but to calculate A->B->D Spark has to start from A again, as the state B is not stored. For computationally expensive transformations we might not have the time or resources to recompute the same operations. 

For this reason, we can store an intermmediate state using the `cache` function. Spark can then start computations from this state instead of the beggining of the transformation chain.

Below is an example showcasing the difference that `cache` makes in computation time. We have data from potato fields in the Netherlands as well as soil consistency data:

In [4]:
#potato data
potatoes = [("Field1", "Miranda", "15-4-2020", 120, "type1"),
            ("Field2", "Miranda", "15-4-2020", 150, "type1"),
            ("Field3", "Miranda", "15-4-2020", 90, "type1"),
            ("Field4", "Bintje", "15-4-2020", 100, "type2"),
            ("Field5", "Bintje", "15-4-2020", 120, "type2"),
            ("Field6", "Felsina", "15-4-2020", 110, "type3"),
            ("Field7", "Felsina", "15-4-2020", 135, "type3"),
            ("Field8", "Felsina", "15-4-2020", 130, "type3"),
            ("Field9", "Felsina", "15-4-2020", 145, "type3"),
            ("Field10", "Fontane", "15-4-2020", 110, "type4")]

#soil data
soil = [("type1", 15, 28, 57), 
        ("type2", 4, 15, 81), 
        ("type3", 22, 32, 46), 
        ("type4", 19, 40, 41), 
        ("type5", 9, 11, 80), 
        ("type6", 9, 17, 74)]

and we want to...

First we convert the data to DataFrames:

In [ ]:
potatoesDF = sqlc.createDataFrame(potatoes, ("Field", "Cultivar", "PlantingDate"))

soilDF = sqlc.createDataFrame(soil, ("ClayPercentage", "SiltPercentage", "SandPercentage"))

### Further reading

* https://spark.apache.org/docs/latest/sql-programming-guide.html
* https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html?highlight=dataframe#pyspark.sql.DataFrame
* https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
* https://spark.apache.org/docs/latest/api/sql/index.html

In [ ]:
q1=[{
        "question": "Spark DataFrame operations are optimized by Spark.",
        "type": "multiple_choice",
        "answers": [
            {
                "code": "True",
                "correct": True
            },
            {
                "code": "False",
                "correct": False
            }
        ]
    }]

q2=[{
        "question": "Spark DataFrames are built on top of RDDs.",
        "type": "multiple_choice",
        "answers": [
            {
                "code": "True",
                "correct": True
            },
            {
                "code": "False",
                "correct": False
            }
        ]
    }]

q3=[{
        "question": "Choose the correct answers:",
        "type": "multiple_choice",
        "answers": [
            {
                "code": "Spark DataFrames are    non-distributed collections of data.",
                "correct": False,
                "feedback": "They are distributed."
            },
            {
                "code": "We can use SQL queries  directly with DataFrames.",
                "correct": True
            },
            {
                "code": "The performance we get  when using DataFrames is programming language dependent.",
                "correct": False,
                "feedback": "DataFrames have the same performance regardless of the language used."
            },
            {
                "code": "When working with       DataFrames we have to carefully think the order of the operations that we want to apply.",
                "correct": True
            }
        ]
    }]

from base64 import b64encode
import json
print(b64encode(bytes(json.dumps(q1), 'utf8')))
print(b64encode(bytes(json.dumps(q2), 'utf8')))
print(b64encode(bytes(json.dumps(q3), 'utf8')))